In [2]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
import ssl
import pdb
import requests
import time
from random import randint
import csv

In [3]:
# parameters of our file
product   = 'B07W8J1GD3'
name    = 'baongoc_dinh' # replace here with your name!
pageNum = 1
url = 'https://www.amazon.com/gp/product/B07W8J1GD3?ref=skillrw_dsk_editorspicks__0'

# # output file
with open('data_' + product + '.txt','w') as fw:
    for p in range(1, pageNum + 1): 

        # tell user which page you're parsing
        print ('Getting page', p)

        # initialize html file to None
        html = None        

        # set URL to get appropriately
        #   if it is the first page
        if p == 1: 
            # url for page 1
            pageLink = url 
        #   if it is tnot the first page
        else: 
            # url for other pages
            pageLink = url + '&pageNumber='+str(p) # make the page url

        # try to scrape times
        for i in range(5): 
            try:
                # get url content
               # access the webpage as Chrome
                my_headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
                #my_headers = { 'User-Agent': 'Mozilla/5.0'}  
                values = {}
                data = urllib.parse.urlencode(values).encode('utf-8')
                req = urllib.request.Request(pageLink, data, my_headers)
                response = urllib.request.urlopen(req)
                html = response.read()
                # if we successuflly got the file, break the loop
                break 
            # requests.get() threw an exception, i.e., the attempt to get the response failed
            except:
                print ('failed attempt #',i)
                # wait 2 secs before trying again
                time.sleep(3)

        if not html:
            # couldnt get the page, ignore
            print('could not get page #', p)
            continue 

        soup = BeautifulSoup(html, 'lxml')
        reviews = soup.find_all('div', {'class': re.compile('a-section review')})

    #   grab the information for each review
        print ('Parsing page',p)
        data=[]
        for review in reviews: 
            rating, date, author, comment,skillname, title, amountofcomments,productid,avgrating,skilldetails,language, description,category,rank,reviewnumber = 'NA', 'NA', 'NA','NA','NA','NA','NA','NA','NA','NA','NA','NA','NA','NA','NA'
        
            # 1. if there is rating information, get it
            ratingChunk = review.find('i', {'data-hook': re.compile('review-star-rating')})
            if ratingChunk: 
                rating = ratingChunk.text.strip()
            # 2. if there is date information, get it
            dateChunk = review.find('span', {'data-hook': re.compile('review-date')})
            if dateChunk: 
                date = dateChunk.text.strip()
                date=date[33:]
            # 3. if there is author name information, get it
            authorChunk = review.find('span', {'class': re.compile('a-profile-name')})
            if authorChunk: 
                author = authorChunk.text.strip()
            # 5. title:
            titleChunk=review.find('a',{'data-hook':'review-title'})
            if titleChunk:
                title=titleChunk.text.strip()
            # 4. if there is review information, get it
            commentChunk = review.find('div', {'data-hook': re.compile('review-collapsed')})
            if commentChunk: 
                comment= commentChunk.text.strip()
            #amount of customer ratings
            amountofratings1=soup.find('span',{'class':'a-size-small a-color-link a2s-review-star-count'})
            if amountofratings1:
                amountofratings=amountofratings1.text.strip()
            #6. How many people wrote a review:
            reviewnumberChunk=soup.find('span',{'data-hook':re.compile('top-customer-reviews-title')})
            if reviewnumberChunk:
                reviewnumber1=reviewnumberChunk.text.strip()
                reviewnumber=reviewnumber1.replace('customer reviews','')
            skill_maincontent=soup.find_all('div',{'class':'a-section a-spacing-none a2s-dp-main-content a2s-responsive-container'})
            for stuff in skill_maincontent:
                skillnameChunk=stuff.find('h1',{'class':'a2s-title-content'})
            if skillnameChunk:
                skillname=skillnameChunk.text.strip()
            amountofreviewsChunk=stuff.find('span',{'class':'a-size-small a-color-link a2s-review-star-count'})
            if amountofreviewsChunk:
                amountofreviews=amountofreviewsChunk.text.strip()
            productid=stuff.find('a',{'class':'a-link-normal a2s-link'})['href']
            productid='amazon.com'+productid
            avgratingSection=soup.find_all('div',{'class':'a2s-responsive-container'})
            for items in avgratingSection:
                avgratingChunk=items.find('span',{'data-hook':'rating-out-of-text'})
                if avgratingChunk:
                    avgrating=avgratingChunk.text.strip()
            description_maincontent=soup.find_all('div',{'class':'a2s-responsive-container'})
            for item in description_maincontent:
            #11. Skill details:
                skilldetailsChunk=item.find('li',{'class':'a-spacing-micro'})
                if skilldetailsChunk:
                    skilldetails1=skilldetailsChunk.text.strip()
                    skilldetails=skilldetails1.replace('\n','')
            #12. Language
                languageChunk=item.find("div",attrs={"class":"a-section","id":"a2s-languages-supported"})
                if languageChunk:
                    language2=languageChunk.text.strip()
                    language1=language2.replace('\n','')
                    language=language1.replace('Supported Languages','')
                    language=language.replace(' ','')
            #Description
            description_maincontent_2=soup.find_all('div',attrs={"class":"a-section","id":"a2s-description"})
            for thing in description_maincontent_2:
                descriptionChunk=thing.find('span')
                if descriptionChunk:
                    description=descriptionChunk.text.strip()
            #Category:
            category=url[61:72]
            rank=url[75]
            data.append([productid,skillname,description,skilldetails,language,category, rank,reviewnumber, amountofratings,avgrating,rating, date, author, title, comment ])
            #write everything to file    		
            fw.write(productid+'\t'+avgrating+'\t'+rating + '\t'+ date +'\t'+ author +'\t'+title+'\t'+ comment+'\t'+skillname+'\t'+amountofratings+'\t'+skilldetails+'\t'+language+'\t'+description+'\t'+category+'\t'+rank+'\t'+reviewnumber+'\n')

Getting page 1
Parsing page 1


In [5]:

data

[]

In [22]:
with open('amazon_skills_review_test.txt', mode='w', encoding='utf-8') as f:
    for statement in data:
        f.write(statement[0] + '\t' + statement[1] + '\t' + statement[2]+'\t'+ statement[3]+'\t'+statement[4]+'\t'+statement[5]+'\t'+statement[6]+'\t'+statement[7]+'\t'+statement[8]+'\t'+statement[9]+'\t'+statement[10]+'\t'+statement[11]+'\t'+statement[12]+'\t'+statement[13]+'\t'+statement[14]+'\n')
with open('amazon_skills_review_test.txt', mode='r', encoding = 'utf-8') as f:
    data = f.read()
# break into lines, then break each line into article_type, aritcle_url, article_text
# throw away last element because it is simply an empty line
data = data.split('\n')[0:-1]

for i in range(0,len(data)):
    data[i] = data[i].split('\t')

In [23]:
# Import pandas to read in data
import numpy as np
import pandas as pd

In [24]:
user_cols=['ID','SkillName','Description','Details','Supported Languages','Category','Rank','#Reviews','Amount of Ratings','Average Product rating', 'User Rating', 'Date', 'Author', 'Title', 'Comment']
df = pd.read_csv("amazon_skills_review_test.txt",sep="\t",header=None,names=user_cols)

In [25]:
df

,ID,SkillName,Description,Details,Supported Languages,Category,Rank,#Reviews,Amount of Ratings,Average Product rating,User Rating,Date,Author,Title,Comment
0,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,5.0 out of 5 stars,"August 10, 2019",zain27,Nice Game !,Fun game to play through the day to take a bre...
1,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,5.0 out of 5 stars,"August 10, 2019",zain27,Nice Game !,Fun game to play through the day to take a bre...
2,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,5.0 out of 5 stars,"August 21, 2019",Tara Rose Customer,just started,think this will be a good interactive skill.
3,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,1.0 out of 5 stars,"February 7, 2020",Randall B.,This particular game Can`t understand me.,"It seems like a fun concept,but I try and arti..."
4,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,2.0 out of 5 stars,"September 20, 2019",D. Marquez,Too Short,Would be good if were able to play more questi...
5,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,4.0 out of 5 stars,"January 30, 2020",Deanna-lynn,😁,Game is great!
6,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,2.0 out of 5 stars,"February 9, 2020",Islander,Wronged correct answers,I answered three correctly but it says I was w...
7,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,1.0 out of 5 stars,"February 6, 2020",Kat,"It could be a fun game, but...",I thought this might be a fun game. First two...
8,amazon.com/product-reviews/B07W8J1GD3,Finish The Phrase,Test your knowledge of famous sayings with the...,This skill contains dynamic content...,"English(AU),English(CA),English(GB),English(IN...",editorspick,0,NaN,214,3.5 out of 5,2.0 out of 5 stars,"October 13, 2019",Ashley E Martin,i love you,You are coool and awesome


In [26]:
df.to_csv('Alexa_Skills_03192020.csv')

In [65]:
skill_maincontent=soup.find_all('div',{'class':'a-section a-spacing-none a2s-dp-main-content a2s-responsive-container'})
for stuff in skill_maincontent:
    skillnameChunk=stuff.find('h1',{'class':'a2s-title-content'})
    if skillnameChunk:
        skillname=skillnameChunk.text.strip()
        print('Skill Name is: ',skillname)
    amountofreviewsChunk=stuff.find('span',{'class':'a-size-small a-color-link a2s-review-star-count'})
    if amountofreviewsChunk:
        amountofreviews=amountofreviewsChunk.text.strip()
        print('Amount of review is: ',amountofreviews)
    productid=stuff.find('a',{'class':'a-link-normal a2s-link'})['href']
    print('Product ID is: ',productid)
#Average rating:
avgratingSection=soup.find_all('div',{'class':'a2s-responsive-container'})
for items in avgratingSection:
    avgratingChunk=items.find('span',{'data-hook':'rating-out-of-text'})
    if avgratingChunk:
        avgrating=avgratingChunk.text.strip()
        print('Average product rating: ', avgrating)
#10. Skill Description:
description_maincontent=soup.find_all('div',{'class':'a2s-responsive-container'})
for item in description_maincontent:
    descriptionChunk=item.find('span')
    if descriptionChunk:
        description1=descriptionChunk.text.strip('\n')
        description='_'.join(description1)
        print('Skill Description: ', description)
#11. Skill details:
    skilldetailsChunk=item.find('li',{'class':'a-spacing-micro'})
    if skilldetailsChunk:
        skilldetails=skilldetailsChunk.text.strip()
        print('Skill details: ', skilldetails)
#12. Language
    languageChunk=item.find("div",attrs={"class":"a-section","id":"a2s-languages-supported"})
    if languageChunk:
        language=languageChunk.text.strip()
        print('supported language: ', language)

Skill Name is:  Finish The Phrase
Amount of review is:  192
Product ID is:  /product-reviews/B07W8J1GD3
Average product rating:  3.8 out of 5
Skill Description:  Test your knowledge of famous sayings with the 'Finish the Phrase' game! Finish five new sayings, slogans, idioms, axioms, puns, proverbs, epithets, and clichés each day to conquer the art of adages. How many of these phrases can you finish?Got feedback? Email finishthephrase@gmail.com
